# FORECAST 911: Predicting 911 Incidents through Weather Forecasts

## Executive Summary:

This is a consulting project for the City of San Diego Fire-Rescue Department done as part of the Insight Data Science Fellows Program. 

### 1. Motivation and Scope of the Project: 

This project focuses specifically on the medical 911 incidents as opposed to the fire-related incidents because medical incidents cover the majority of the 911 calls and the highest cost of response.  Between September 1st 2016 and August 31st 2019, more than 420K 911 medical incidents occurred in the City of San Diego, which corresponds to more than 380 incidents per day. Many of these incidents require multiple ambulances dispatched to the scene. To provide high-quality service is expensive: It costs about \$10K to maintain and staff an ambulance for a day and an ambulance can answer about 16 incidents in 24 hours. On the other hand, if there are not enough ambulances available to respond to an incident, the longer response time can make loss of life more likely.

Therefore, the City's Fire-Rescue department would like to improve response times and staff people and ambulances more efficiently. To do so, they need to have an idea of about the volume 911 activity every day. 

Up to now, they have been using a univariate time series forecasting model which only uses historical incident data to predict the future number of 911 calls. However, there might be some external features which can be added to this kind of univariate models to increase prediction accuracy. For example, weather-related features might be important in predicting the number of 911 calls:

This project explores which weather-related features, if any, might be important in predicting the daily number 911 calls in the City of San Diego and uses these features to run a multivariate time-series forecasting model and shows the improvement in the prediction accuracy and the potential financial savings in ambulance costs as a result.

### 2. Data Description: 

The City of San Diego provided three years of 911 incident data from September 1st 2016 to August 31st 2019 which has 24 columns. The following features are available in the data set:

**1. ID:** Ordinal categorical. Integer. The ID increases in the chronological order of incidents. 7-digit unique ID for each incident. If more than one ambulance is sent to the scene, the same incident number appears in multiple rows.

**2. Master_Incident_Number:** Ordinal categorical. Object. "FS"+"2-digit year"+"6-digit incident number starting from 000001 at the start of each year and increases in the chronological order of incidents". If more than one ambulance is sent to the scene, the same incident number might appear in multiple rows.

**3. Response_Date:** Ordinal categorical. Object. Date and time of when the 911 call is received. The format is YYYY-MM-DD HH:MM:SS.000.

**4. Jurisdiction:** Nominal categorical. Object. This feature shows the jurisdiction of the incident. This feature only contains the string "San Diego" for this data set.  

**5. Incident_Type:**  Nominal categorical. Object. This feature shows 28 different incident types as shown above. Medical Aid 1 shows most acute incidents, Medical Aid 4 shows least acute incidents.

**6. Problem:**  Nominal categorical. Object. This feature shows 218 different problems associated with the incidents as shown above. The most frequent problem is described as "Sick Person (Specific Dx)(L1)". L1 indicates most acute version of chest pain while L4 represents the least acute version.

**7. Priority_Description:** Nominal categorical. Object. This feature shows 27 priority descriptions for the incidents as shown above. The lower numbers in the descriptions represent more acute incidents. 

**8. Call_Category:**     Nominal categorical. Object. This feature shows 3 categories for the medical 911 calls. From most frequent to least frequent: Emergency Medical Response, Non-Emergency Medical Response, Urgent Medical Response.

**9. Transport_Mode:** Nominal categorical. Object. This feature shows categories for the transport mode of incidents such as: 50-Non Emergency, 40-BLS Status Transport, 30-IV/No Medication, 20-IV/Medication, 10-Acute/Medical Trauma, MUTUAL AID TRANSPORT. The lower numbers represent more acute incidents. 

**10. Location_Name:** Nominal categorical. Object. This feature shows the description of the location of the incident such as hotel, school, residence, etc.  

**11. Address:** Nominal categorical. Object. This feature shows the address of the incident.

**12. Apartment:** Nominal categorical. Object. This feature shows the apartment number of the incident.

**13. Postal_Code:** Nominal categorical. Float. This feature shows the zipcode of the incident.

**14. Longitude:** Numeric. Float. This feature shows the longitude of the incident as an 9-digit number.

**15. Latitude:** Numeric. Float. This feature shows the latitude of the incident as an 8-digit number.

**16. Longitude_Decimal:** Numeric. Float. This feature shows the latitude of the incident as a number with 6 decimals.

**17. Latitude_Decimal:**  Numeric. Float. This feature shows the latitude of the incident as a number with 6 decimals.

**18. Cross_Street:** Nominal categorical. Object. This feature shows the cross street of where the incident occurred.

**19. MethodOfCallRcvd:** Nominal categorical. Object. This feature shows the method of the 911 call received such as through a cell phone, alarm company, etc.    

**20. Time_First_Unit_Assigned:** Ordinal categorical. Object. Date and time of when the first unit is assigned to the incident. The format is YYYY-MM-DD HH:MM:SS.000.

**21. Time_First_Unit_Enroute:** Ordinal categorical. Object. Date and time of when the first unit is enroute to the incident scene. The format is YYYY-MM-DD HH:MM:SS.000.

**22. Time_First_Unit_Arrived:** Ordinal categorical. Object. Date and time of when the first unit arrives at the incident scene. The format is YYYY-MM-DD HH:MM:SS.000.

**23. Call_Disposition:** Nominal categorical. Object.  The disposition method of the 911 call. 

**24. TimeFirstStaged:** Ordinal categorical. Object. Date and time of when the ambulances are first staged while police secure potentially dangerous incident scene. Null if not staged.
The format is YYYY-MM-DD HH:MM:SS.000.

National Weather Service includes 16 weather stations related to San Diego: https://www.weather.gov/sgx/cliplot
For each of the weather stations and for each month of the year, daily observed precipitation, low and high temperatures are provided in tables. The complete set of features in these tables are: Date, Observed Low (F), Observed High (F), Normal Low (F), Normal High (F), Record Low (F), Year, Record High (F), Year, Observed Precipitation (inches), Record Precipitation (inches) and Year. We will focus on Observed High: the highest temperature observed during the day, Observed Low: the lowest temperature observed during the day and Observed Precipitation: observed precipitation during the day.

Before collecting this data, using the coordinates of the weather stations and the coordinates of each incident in the data set, I determined which weather stations are most relevant to the locations of incidents and dropped the ones that are not the closest weather station to more than 10 incidents. I collected the daily weather data over three years for the remaining 5 weather stations: San Diego Lindbergh Field, San Diego Montgomery Field, San Diego Brown Field, Ramona and Carlsbad.

### 3. Feature Engineering, ML Algorithms:

#### 3.1. Single-Weather-Station Case:
In this case, I used only the weather data from San Diego Lindbergh field weather station, which is the closest weather station to the most number of incidents in San Diego, and all incident data.

I engineered four weather-related features using features 'observed_high', 'observed_low' and 'observed_precipitation' in the weather-station data:

1. **high_low_diff:** The difference between the highest and lowest temberatures observed during the day: df\['high_low_diff'\]=df\['observed_high'\]-df\['observed_low'\]
2. **high_temp_diff:**  The highest temperature on day 't' minus the highest temperature on day 't-1': df\['high_temp_diff'\]=df\['observed_high'\].diff()
3. **high_temp_alert:** df\['high_temp_alert'\]=1 if the difference with the highest temperature on the previous day is higher than or equal to 10 F degrees, otherwise 0. 
4. **rain_alert:**'df\['rain_alert'\]=1 if no precipitation is observed on the previous day and there is positive precipitation today.

Along with the 'observed_low', 'observed_high' and 'observed_precipitation' data, there are seven weather-related features to be evaluated in terms of feature importances.

On top of these features, I added seven time lags df\[t-1\] through df\[t-7\] where df\[t-i\] is the number of incidents i days before day t. 

Comparing the importance of these 14 features in predicting the number of incidents by three supervised learning algorithms, XGBoost, AdaBoost and RandomForest Regressor, the most important weather-related feature seems to 'observed_high' temperature.
I reached this result after checking the correlations among features and removing 'observed_high' and 'observed_low' features from the feature list and running feature importances again to see if some other important feature might be shadowed by their correlation to these features.

#### 3.2. Multiple-Weather-Station Case: 

I ran the feature importances for five weather stations separately using the corresponding weather station data and the corresponding incident data. For San Diego Lindbergh Field, San Diego Brown Field and Ramona stations, the most important feature is again the 'observed_high' temperature while for San Diego Montgomery Field, the most important weather-related feature is 'high_low_diff', the difference between highest and lowest temperatures during the day. Finally, for Carlsbad, the most important weather-related feature is 'high_temp_diff', the difference between highest temperatures today and yesterday. However, it should be noted that while San Diego Lindbergh Field and San Diego Brown Field incident data show some correlation with weather-related features, the incident data assigned to other weather stations do not have a strong correlation with weather-related features.


### 4. Time Series Forecasting:

#### 4.1. Univariate and Multivariate Time Series Forecasting with Single-Weather-Station Data:

First, I run a univariate SARIMA model using all incident data and compare this model to a multivariate SARIMAX model using the 'observed_high' temperature data from San Diego Lindbergh field weather station and all incident data. The RMSE (root-mean-squared error) of the model drops from 27.375 to 22.713 when we switch from the univariate model to the multivariate model. Note that the average number of medical incidents in San Diego is approximately 380 per day and the reduction in RMSE means that our predictions are on average about 5 incidents closer to the observed number of incidents every day. 

#### 4.2. Univariate and Multivariate Time Series Forecasting with Multiple-Weather-Station Data:

First, I run a univariate SARIMA model using incident data assigned to each of the five weather stations separately and compare each of these models to a multivariate SARIMAX model using the most important weather-related feature for each weather station and the corresponding incident data. The RMSE (root-mean-squared error) of the resulting predictions drops from 23.628 to 22.392 when we switch from the univariate model to the multivariate model. This result indicates that the univariate models' performance increases when we divide the data into 5 sets and fit 5 separate models to the data. Also, running feature importances separately for each weather station and fitting 5 separate multi-variate models does not increase the performance of the resulting predictions significantly.

### 5. Results

The highest temperature during the day is an important weather-related feature which can be included in the prediction of daily number of medical 911 incidents. It is sufficient to only collect this weather data from San Diego Lindbergh Field weather station and run a multivariate SARIMAX model to increase the performance of the model significantly. 

When we compare the performance of this multivariate model with single-weather-station data to the performance of five separate models collecting data from five separate weather stations, there is no significant improvement in the model performance. Therefore, it is sufficient to run a single model and collect data only from one weather station to achieve a 17\% improvement in the performance in terms of the reduction in RMSE.

### 6. Managerial Insights 

Using the highest temperature during the day at San Diego Lindbergh Field weather station, our predictions can be on average 4.662 incidents closer to the observed number of incidents every day. This corresponds to approximately 1687 incidents per year. Assuming 1) one ambulance can respond to 16 incidents in 24 hours, 2) it costs 10K to maintain an ambulance for 24 hours, and 3) the cost of underestimating and overestimating the number of incidents costs the same, predicting 1687 incidents more accurately would correspond to potential annual cost savings of more than $1M. Note that underestimating the number of incidents can increase the probability of loss of life due to the increase in response time and also result in significant overtime costs. This cost is likely to be higher than the cost of overestimating the number of incidents.